<a href="https://colab.research.google.com/github/shivamgoswami2711/run-your-gpt/blob/main/Gstudio_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 🎵 Run Silent Audio Player { display-mode: "form" }

#@markdown 👇 Press play on the audio player that appears below. This will keep the Colab tab alive and prevent Google from disconnecting you for inactivity.
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#@title ##**🚀 Gstudio AI API's**

import sys, os, sys, base64, subprocess, json, shutil, requests, time, pathlib, multiprocessing
from IPython.display import clear_output, display, HTML
from IPython.utils import capture
from google.colab import files, drive
from PIL import Image

#PARAMS
#@markdown 👈 Press this button to start the installation process. The links and any further instructions will appear at the bottom after a few minutes.

model_repo_download = "TheBloke/MythoMax-L2-13B-GPTQ"
single_file_download = ""
launch_arguments = ""
save_to_google_drive = "off"
verbose = True
multi_user = True
api = True
superbooga = False
google_translate = False
long_replies = False
character_bias = False
silero_tts = False
elevenlabs_tts = False
whisper_stt = False
send_pictures = False
gallery = False
sd_api_pictures = False
openai = False
ngrok = False

settings_file = " "
perplexity_colors = False
xformers = True
deepspeed = False
auto_devices = True
cpu = False
no_cache = False
precision = "default"
wbits = "default"
groupsize = "default"


trust_remote_code = True
share = True
run_web_ui = True
model = ""

# Stop program if both model input fields are empty
# You WILL download the model, and you will be happy
if (model_repo_download == " ".strip() and single_file_download == " ".strip()):
  print(f"\033[92m\n\n######################################################\n\nNo model selected! Please select a model above, then run this cell again.\n\n######################################################\n\n\033[0m")
  sys.exit()

#Install ooba
def install_ooba():
  global launch_arguments
  if os.path.exists(repo_dir):
    %cd {repo_dir}
    !git pull
  else:
    !git clone https://github.com/oobabooga/text-generation-webui.git
  if ("chatlogs and characters" in save_to_google_drive):
    if not os.path.exists(f"{base_drive_dir}/oobabooga-data"):
      os.mkdir(f"{base_drive_dir}/oobabooga-data")
    if not os.path.exists(f"{base_drive_dir}/oobabooga-data/logs"):
      os.mkdir(f"{base_drive_dir}/oobabooga-data/logs")
    if not os.path.exists(f"{base_drive_dir}/oobabooga-data/characters"):
      shutil.move("text-generation-webui/characters", f"{base_drive_dir}/oobabooga-data/characters")
    else:
      !rm -r "text-generation-webui/characters"

    !ln -s "$base_drive_dir/oobabooga-data/logs" "text-generation-webui/logs"
    !ln -s "$base_drive_dir/oobabooga-data/characters" "text-generation-webui/characters"
  else:
    !mkdir text-generation-webui/logs
  !ln -s text-generation-webui/logs .
  !ln -s text-generation-webui/characters .
  !ln -s text-generation-webui/models .
  %rm -r sample_data
  %cd {repo_dir}
  if not (" ".strip() in settings_file):
    !wget {settings_file} -O settings-template.yaml
    launch_arguments.add(f'--settings settings-template.yaml')
  !pip install -r requirements.txt | grep -v 'already satisfied'
  print(f"\033[1;32;1m\nIf you see a warning about packages, just ignore it. There is no need to restart the runtime.\n\033[0;37;0m")
  # Install extension req
  if (deepspeed) or ('deepspeed' in launch_arguments):
    !pip install -U mpi4py | grep -v 'already satisfied'
    !pip install -U deepspeed | grep -v 'already satisfied'
  if (xformers) or ('xformers' in launch_arguments):
    !pip install xformers | grep -v 'already satisfied'
  if (api) or ('api' in launch_arguments):
    !pip install -r extensions/api/requirements.txt | grep -v 'already satisfied'
  if (google_translate) or ('google_translate' in launch_arguments):
    !pip install -r extensions/google_translate/requirements.txt | grep -v 'already satisfied'
  if (superbooga) or ('superbooga' in launch_arguments):
    !pip install -r extensions/superbooga/requirements.txt | grep -v 'already satisfied'
  if (silero_tts) or ('silero_tts' in launch_arguments):
    !pip install -r extensions/silero_tts/requirements.txt | grep -v 'already satisfied'
  if (elevenlabs_tts) or ('elevenlabs_tts' in launch_arguments):
    !pip install -r extensions/elevenlabs_tts/requirements.txt | grep -v 'already satisfied'
  if (whisper_stt) or ('whisper_stt' in launch_arguments):
    !pip install -r extensions/whisper_stt/requirements.txt | grep -v 'already satisfied'
  if (openai) or ('openai' in launch_arguments):
    !pip install -r extensions/openai/requirements.txt | grep -v 'already satisfied'
  if (ngrok) or ('ngrok' in launch_arguments):
    !pip install -r extensions/ngrok/requirements.txt | grep -v 'already satisfied'


#Mount Google Drive
if not ("off" in save_to_google_drive):
  if ("chatlogs and characters" in save_to_google_drive):
    drive.mount('/content/drive')
    base_drive_dir = "/content/drive/MyDrive/"
    repo_dir = '/content/text-generation-webui'
    %cd /content
    install_ooba()
  if ("chatlogs, characters, and models" in save_to_google_drive):
    drive.mount('/content/drive')
    base_drive_dir = "/content/drive/MyDrive/"
    repo_dir = '/content/drive/MyDrive/text-generation-webui'
    model_dir = '/content/drive/MyDrive/text-generation-webui/models'
    %cd /content/drive/MyDrive
    install_ooba()
else:
  %cd /content
  repo_dir = '/content/text-generation-webui'
  model_dir = '/content/text-generation-webui/models'
  install_ooba()

# Repo download
def repo_download():
  global model
  model = model_repo_download
  %cd {repo_dir}
  !python download-model.py {model}
  model = model.replace('/', '_')

if (model_repo_download) and not (single_file_download):
  clear_output(wait = False)
  repo_download()

# Single file download
def single_download():
  global model
  def get_filename_from_url(single_file_download):
    return os.path.basename(single_file_download)
  model = get_filename_from_url(single_file_download)
  %cd {model_dir}
  !apt install aria2
  !aria2c -x 16 -s 16 -o {model} {single_file_download}
  %cd {repo_dir}
  # Reinstall llama-cpp-python with BLAS for GGML GPU support and add llama.cpp flags
  !pip uninstall -y llama-cpp-python
  !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir

if (single_file_download) and not (model_repo_download):
  clear_output(wait = False)
  single_download()

# If both input fields are filled
if (model_repo_download and single_file_download):
  clear_output(wait = False)
  valid_input = False
  while not valid_input:
    choice = input("\n\n######################################################\n\nYou have selected two models at once. Choose one to download.\n\nIn the box below, type 1 for the repo download, or type 2 for the single file download. Then press Enter.\n\n")
    choice = int(choice)
    if choice == 1:
      valid_input = True
      repo_download()
    elif choice == 2:
      valid_input = True
      single_download()
    else:
      print("\nInvalid input.\n")

# Arguments
launch_arguments = set()
if ('GPTQ' in model or 'gptq' in model): launch_arguments.add('--loader exllama')
if ('.bin' in model or 'GGML' in model or 'ggml' in model): launch_arguments.add('--threads 16 --n-gpu-layers 20 --loader llama.cpp')
if not ('default' in wbits): launch_arguments.add(f'--wbits {wbits}')
if not ('default' in groupsize): launch_arguments.add(f'--groupsize {groupsize}')
if ('8bit' in precision): launch_arguments.add('--load-in-8bit')
if ('4bit' in precision): launch_arguments.add('--load-in-4bit')
if trust_remote_code: launch_arguments.add('--trust-remote-code')
if multi_user: launch_arguments.add('--multi-user')
if verbose: launch_arguments.add('--verbose')
if no_cache: launch_arguments.add('--no-cache')
if xformers: launch_arguments.add('--xformers')
if deepspeed: launch_arguments.add('--deepspeed')
if api: launch_arguments.add('--api --public-api')
if share: launch_arguments.add('--share')
if auto_devices: launch_arguments.add('--auto-devices')
if cpu: launch_arguments.add('--cpu')

#Extension toggles
active_extensions = []
if long_replies: active_extensions.append('long_replies')
if send_pictures: active_extensions.append('send_pictures')
if character_bias: active_extensions.append('character_bias')
if google_translate: active_extensions.append('google_translate')
if superbooga: active_extensions.append('superbooga')
if silero_tts: active_extensions.append('silero_tts')
if elevenlabs_tts: active_extensions.append('elevenlabs_tts')
if whisper_stt: active_extensions.append('whisper_stt')
if gallery: active_extensions.append('gallery')
if openai: active_extensions.append('openai')
if sd_api_pictures: active_extensions.append('sd_api_pictures')
if ngrok: active_extensions.append('ngrok')
if perplexity_colors: active_extensions.append('perplexity_colors')

# If any extensions are selected:
# Append the --extensions flag and all selected extensions
if len(active_extensions) > 0:
  launch_arguments.add(f'--extensions {" ".join(active_extensions)}')

clear_output(wait = True)

# Run WebUI
print(f"\033[1;32;1m\n######################################################\n\nThe model should load in about a minute. To enter TextGen, click on the link below that ends with gradio.live.\n\nFor SillyTavern users, copy the \"non-streaming URL\" (ends with \"/api\") and paste it into the \"Blocking API URL\" in the API settings.\n\nYou may get a \"Could not start cloudflared\" error that prevents you from receiving the API link. If you do, just stop this cell and run it again.\n\n######################################################\n\033[0;37;0m")
if ('deepspeed' in launch_arguments):
  cmd =f"deepspeed --num_gpus=1 server.py --model {model} {' '.join(launch_arguments)}"
  print(cmd)
  !$cmd
else:
  cmd = f"python server.py --model {model} {' '.join(launch_arguments)}"
  print(cmd)
  !$cmd

/content/text-generation-webui

######################################################

The model should load in about a minute. To enter TextGen, click on the link below that ends with gradio.live.

For SillyTavern users, copy the "non-streaming URL" (ends with "/api") and paste it into the "Blocking API URL" in the API settings.

You may get a "Could not start cloudflared" error that prevents you from receiving the API link. If you do, just stop this cell and run it again.

######################################################

python server.py --model TheBloke_MythoMax-L2-13B-GPTQ --trust-remote-code --xformers --loader exllama --auto-devices --multi-user --api --public-api --verbose --share
2023-08-19 14:12:05 WARNING:trust_remote_code is enabled. This is dangerous.
2023-08-19 14:12:05 WARNING:The gradio "share link" feature uses a proprietary executable to create a reverse tunnel. Use it with care.
2023-08-19 14:12:05 WARNING:The multi-user mode is highly experimental. DO NOT EXP